In [1]:
pip install git+ssh://git@github.com/dorianbrown/rank_bm25.git

Note: you may need to restart the kernel to use updated packages.  Cloning ssh://****@github.com/dorianbrown/rank_bm25.git to c:\users\josipk~1\appdata\local\temp\pip-req-build-dxq56k9f

  Created wheel for rank-bm25: filename=rank_bm25-0.2.1-py3-none-any.whl size=8574 sha256=50aa8694584d2362757a04cba4cbda1973ffcccceb9791bdb4dcd0092bad2b47
  Stored in directory: C:\Users\JOSIPK~1\AppData\Local\Temp\pip-ephem-wheel-cache-ccyrioxm\wheels\b5\04\7a\a7a2d5c6113841f0d3bd39f9671cc7479f62bd33e2b63464d0
Successfully built rank-bm25


In [9]:
from rank_bm25 import BM25Okapi
import glob
import os
import numpy as np
import string

In [47]:
## Importing bad and good bias words
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\Working on at the moment\IR Project')
bad_bias_words = open("BadBias.txt").read().split("\n")
good_bias_words = open("GoodBias.txt").read().split("\n")

## Initialzing needed dictionaries for single and paired words
bad_bias_grams = {}
bad_bias_bigrams = {}
good_bias_grams = {}
good_bias_bigrams = {}

## Adding bad bias words from basic text to dictionaries
for x in bad_bias_words:
    if(len(x.strip().split(" ")) == 2):
        bad_bias_bigrams[x] = 0
    else:
        bad_bias_grams[x] = 0

## Adding good bias words from basic text to dictionaries
for x in good_bias_words:
    if(len(x.strip().split(" ")) == 2):
        good_bias_bigrams[x] = 0
    else:
        good_bias_grams[x] = 0

## Importing movie scripts
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\Working on at the moment\IR Project\data')
myFiles = glob.glob('*.txt')
scripts = []
for file in myFiles:
    file = open(file,mode='r', encoding='utf-8-sig')
    scripts.append(file.read())
    file.close()

In [48]:
## Text Preprocessing

## Removing unneded white spaces and splitting scipts into tokens
scripts = [" ".join(doc.split()) for doc in scripts]
scripts = [doc.lower() for doc in scripts]
scripts = [doc.translate(str.maketrans("", "", string.punctuation)) for doc in scripts]
tokenized_corpus = [doc.split(" ") for doc in scripts]

## Creating pairs of tokens
tokenized_corpus_bigrams = []
for file in tokenized_corpus:
    current = []
    for i in range(len(file) - 1):
        current.append(file[i] + " " + file[i+1])
    tokenized_corpus_bigrams.append(current)

In [49]:
## Counting good and bad bias for each script and creating a weight list for all scripts
## Weight consists of number of different good/bad biases and the amount of times they occur (only second implemented for now)
bias_list = []
for i in range(len(scripts)):
    ## Increment dictionaries
    for word in tokenized_corpus[i]:
        if word in bad_bias_grams:
            bad_bias_grams[word] += 1
        elif word in good_bias_grams:
            good_bias_grams[word] += 1
    for word in tokenized_corpus_bigrams[i]:
        if word in bad_bias_bigrams:
            bad_bias_bigrams[word] += 1
        elif word in good_bias_bigrams:
            good_bias_bigrams[word] += 1
    
    ## Count bias and add to list of biases
    bad_bias = sum(bad_bias_grams.values()) + sum(bad_bias_bigrams.values())
    good_bias = sum(good_bias_grams.values()) + sum(good_bias_bigrams.values())
    bias = good_bias - bad_bias
    bias_list.append(bias)
    
    ## Reset dictionaries to zero        
    bad_bias_grams = bad_bias_grams.fromkeys(bad_bias_grams, 0)
    bad_bias_bigrams = bad_bias_bigrams.fromkeys(bad_bias_bigrams, 0)
    good_bias_grams = good_bias_grams.fromkeys(good_bias_grams, 0)
    good_bias_bigrams = good_bias_bigrams.fromkeys(good_bias_bigrams, 0)

## Print bias_list for better visualization
bias_list = [x/50 for x in bias_list]
bm25 = BM25Okapi(tokenized_corpus)

In [50]:
## Printing movie scripts ranked from most to least biased
bias = {}
for i in range(len(myFiles)):
    bias[myFiles[i]] = bias_list[i]

for w in sorted(bias, key=bias.get, reverse=False):
    print(w, bias[w])

Script_Wizard of Oz, The.txt -78.86
Script_Lord of the Rings_ Return of the King.txt -61.52
Script_Bourne Supremacy, The.txt -59.1
Script_Boogie Nights.txt -53.84
Script_Gone in 60 Seconds.txt -48.36
Script_Bourne Identity, The.txt -43.44
Script_At First Sight.txt -39.4
Script_Awakenings.txt -37.5
Script_Cliffhanger.txt -35.12
Script_Star Trek.txt -33.48
Script_Get Shorty.txt -32.18
Script_44 Inch Chest.txt -31.78
Script_Absolute Power.txt -30.84
Script_Island, The.txt -30.82
Script_Heavenly Creatures.txt -30.22
Script_Star Wars_ The Phantom Menace.txt -29.24
Script_Taking of Pelham One Two Three, The.txt -28.82
Script_King Kong.txt -28.68
Script_Manchurian Candidate, The.txt -27.92
Script_Tomorrow Never Dies.txt -27.4
Script_Wolf of Wall Street, The.txt -27.18
Script_Charade.txt -25.9
Script_Avengers, The.txt -25.48
Script_Road, The.txt -25.48
Script_Rear Window.txt -25.36
Script_Apocalypse Now.txt -24.94
Script_25th Hour.txt -24.16
Script_Magnolia.txt -24.02
Script_Eagle Eye.txt -23.

Script_Observe and Report.txt -2.46
Script_Wild Things_ Diamonds in the Rough.txt -2.46
Script_Back-up Plan, The.txt -2.42
Script_Elephant Man, The.txt -2.42
Script_Imaginarium of Doctor Parnassus, The.txt -2.42
Script_Remember Me.txt -2.42
Script_Get Low.txt -2.4
Script_Hard Rain.txt -2.4
Script_Sixth Sense, The.txt -2.4
Script_I'll Do Anything.txt -2.38
Script_Iron Lady, The.txt -2.38
Script_Ninth Gate, The.txt -2.36
Script_50-50.txt -2.34
Script_El Mariachi.txt -2.34
Script_Mr Brooks.txt -2.34
Script_What Lies Beneath.txt -2.34
Script_Pandorum.txt -2.32
Script_Prophecy, The.txt -2.32
Script_Amelia.txt -2.3
Script_Garden State.txt -2.3
Script_Lake Placid.txt -2.24
Script_Raising Arizona.txt -2.24
Script_Assassins.txt -2.22
Script_Jimmy and Judy.txt -2.22
Script_Little Athens.txt -2.2
Script_17 Again.txt -2.18
Script_Croods, The.txt -2.18
Script_Hellraiser_ Hellseeker.txt -2.16
Script_Kung Fu Panda.txt -2.16
Script_Star Wars_ Return of the Jedi.txt -2.16
Script_Strangers on a Train.tx

In [51]:
## Importing queries
################################### CHANGE ROUTE FOR CODE TO WORK ####################################################
os.chdir(r'C:\Users\JosipKoprcina\Desktop\Working on at the moment\IR Project')
queries = open("Queries.txt").read().split("\n")

tokenized_queries = []
for query in queries:
    tokenized_queries.append(query.split(" "))

In [53]:
doc_scores = bm25.get_scores(tokenized_queries[1])
doc_scores_bias = [doc_scores[i] + bias_list[i] for i in range(len(doc_scores))]

movies = {}
movies_bias = {}
for i in range(len(myFiles)):
    movies[myFiles[i]] = doc_scores[i]
    movies_bias[myFiles[i]] = doc_scores_bias[i]

In [54]:
#Old way
#bm25.get_top_n(tokenized_query, myFiles, n=3)

In [55]:
## Printing movie order of relevance to query before bias was added
for w in sorted(movies, key=movies.get, reverse=True):
    print(w, movies[w])

Script_Kill Bill Volume 1 & 2.txt 7.086971894953813
Script_Mulan.txt 7.0789045679624785
Script_Conan the Barbarian.txt 7.018520838751684
Script_Robin Hood_ Prince of Thieves.txt 6.990643525121721
Script_Scott Pilgrim vs the World.txt 6.983193158283482
Script_Erik the Viking.txt 6.896960557957604
Script_Flash Gordon.txt 6.859838111724288
Script_Clash of the Titans.txt 6.823373747536736
Script_Book of Eli, The.txt 6.814563067301217
Script_Snow White and the Huntsman.txt 6.801246944693855
Script_Serenity.txt 6.7483769257240525
Script_G.I. Joe_ The Rise of Cobra.txt 6.745954991286453
Script_Four Rooms.txt 6.741306855387087
Script_Crouching Tiger, Hidden Dragon.txt 6.7032973443386155
Script_Brazil.txt 6.669436552905941
Script_Christ Complex.txt 6.6679489993695205
Script_Sleepy Hollow.txt 6.6679034369853065
Script_Gremlins.txt 6.633960812809817
Script_Blade.txt 6.623542440948406
Script_Red Riding Hood.txt 6.613622368013532
Script_Heavy Metal.txt 6.606479251600472
Script_Mummy, The.txt 6.6007

Script_Hudsucker Proxy, The.txt 3.119591905956419
Script_Red Planet.txt 3.1185574042729116
Script_Producer, The.txt 3.11756883924092
Script_Death to Smoochy.txt 3.1175110420505106
Script_Twilight_ New Moon.txt 3.115721781271052
Script_Basquiat.txt 3.113494576043484
Script_Above the Law.txt 3.1134920583588315
Script_Feast.txt 3.111834179342496
Script_Getaway, The.txt 3.1063667499735144
Script_Gods and Monsters.txt 3.1058675292206592
Script_Sense and Sensibility.txt 3.10382278233197
Script_Inventing the Abbotts.txt 3.10355324292937
Script_Arbitrage.txt 3.101598231295008
Script_Panic Room.txt 3.0999092608164642
Script_Walk to Remember, A.txt 3.099425498371985
Script_Hostage.txt 3.097484689597521
Script_Megamind.txt 3.096809665053592
Script_Final Destination.txt 3.094845826904475
Script_Platoon.txt 3.094213378267439
Script_Mission to Mars.txt 3.0940605190089987
Script_Very Bad Things.txt 3.0871390560065395
Script_Man Who Knew Too Much, The.txt 3.0839642466424326
Script_Labor of Love.txt 3.

Script_Pi.txt 2.5075439144310865
Script_Who Framed Roger Rabbit_.txt 2.5053546959091513
Script_Collateral.txt 2.5031999809853422
Script_Air Force One.txt 2.5028369003452378
Script_Simone.txt 2.5017223823640635
Script_Lake Placid.txt 2.4952282934273033
Script_Insider, The.txt 2.4850718050021667
Script_Bodies, Rest & Motion.txt 2.4808146395250152
Script_Dark City.txt 2.4806826031836477
Script_Fletch.txt 2.4768086087319605
Script_Day the Earth Stood Still, The.txt 2.4757759336669674
Script_Chasing Sleep.txt 2.4694787892766916
Script_Affliction.txt 2.456896004714805
Script_Suspect Zero.txt 2.450063944678433
Script_JFK.txt 2.4449670312414913
Script_Visitor, The.txt 2.4317807218610885
Script_Eight Legged Freaks.txt 2.4281143899940507
Script_Fargo.txt 2.4240941726293417
Script_Phone Booth.txt 2.417870927171005
Script_Ace Ventura_ Pet Detective.txt 2.4177257366185354
Script_Lock, Stock and Two Smoking Barrels.txt 2.397355590679313
Script_Wolf of Wall Street, The.txt 2.3954494773425425
Script_P

In [56]:
## Printing movie order of relevance to query after bias was added
for w in sorted(movies_bias, key=movies_bias.get, reverse=True):
    print(w, movies_bias[w])

Script_Christ Complex.txt 6.80794899936952
Script_Mulan.txt 6.718904567962478
Script_Chronicles of Narnia_ The Lion, the Witch and the Wardrobe.txt 6.323420037827665
Script_Indiana Jones IV.txt 5.9420059728313666
Script_Aladdin.txt 5.6624535548243555
Script_Erik the Viking.txt 5.496960557957603
Script_Slumdog Millionaire.txt 5.369134982983205
Script_Heathers.txt 5.331359248981559
Script_Gremlins.txt 5.053960812809817
Script_Frozen.txt 4.98042521563538
Script_Year One.txt 4.927379413670723
Script_Tristan and Isolde.txt 4.771365220094757
Script_Highlander.txt 4.741108033206138
Script_Pulp Fiction.txt 4.656857130153871
Script_Supergirl.txt 4.561616779591582
Script_Sister Act.txt 4.541032070544201
Script_Scott Pilgrim vs the World.txt 4.463193158283483
Script_Sessions, The.txt 4.24485183721049
Script_Hebrew Hammer, The.txt 4.223598125408371
Script_Gangs of New York.txt 4.1688582993548255
Script_Happy Birthday, Wanda June.txt 4.074508608069484
Script_Ferris Bueller's Day Off.txt 3.952096916

Script_12 Monkeys.txt -4.626794383602853
Script_Game, The.txt -4.656735293877068
Script_Jerry Maguire.txt -4.666118302211906
Script_8MM.txt -4.671287614674107
Script_Barry Lyndon.txt -4.68117828115175
Script_Deer Hunter, The.txt -4.682154636822187
Script_Cincinnati Kid, The.txt -4.695770595000378
Script_Shampoo.txt -4.699138122501512
Script_G.I. Jane.txt -4.722711338831209
Script_12.txt -4.72888512913549
Script_Adjustment Bureau, The.txt -4.733904879371881
Script_Sideways.txt -4.756647442141027
Script_Long Kiss Goodnight, The.txt -4.765424146771393
Script_Candle to Water.txt -4.78008557190459
Script_Fear and Loathing in Las Vegas.txt -4.780889371298037
Script_Burlesque.txt -4.790816349791445
Script_Kundun.txt -4.82
Script_Blue Velvet.txt -4.827042067435061
Script_Cradle 2 the Grave.txt -4.85006144454527
Script_Whistleblower, The.txt -4.862928266912954
Script_American Splendor.txt -4.882865168548955
Script_This Boy's Life.txt -4.887557784584102
Script_Men in Black.txt -4.890450837472486